In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
train= pd.read_csv('/kaggle/input/examen-ml2/train.csv', engine = 'python' , encoding = 'utf8', sep=';')
test = pd.read_csv('/kaggle/input/examen-ml2/test.csv', engine = 'python' , encoding = 'utf8', sep=';')

In [4]:
train['Target'].unique()

In [5]:
from sklearn.linear_model import LogisticRegression

unique_targets = train['Target'].unique()
unique_targets.sort()

models = {}
# variables de Train sans ID et Target
features = train.columns.drop(['Student ID','Target'])

for target in unique_targets:
    model = LogisticRegression(solver='liblinear')
    
    X_train = train[features]
    y_train = train['Target'] == target
    
    # on adapte les données au modèle
    model.fit(X_train, y_train)
    models[target] = model

In [7]:
# on teste sur le set de test
testing_probs = pd.DataFrame(columns=unique_targets)

for origin in unique_targets:
    X_test = test[features]
    
    # on calcule les probabilités qu'un étudiant ait tel ou tel résultat
    testing_probs[origin] = models[origin].predict_proba(X_test)[:,1]
    

testing_probs.head()

In [8]:
# on a obtenu les probas pour chaque étudiant 
# puis on choisit la proba la plus élevée avec idmax

predicted_origins = testing_probs.idxmax(axis = 1)

# on affiche les 5 premières valeurs et on compare
print(predicted_origins[0:5])
testing_probs

In [15]:
# on ajoute la colonne des prédictions au set de test
test['prediction'] = predicted_origins

In [23]:
final_df = test[['Student ID','prediction']]
final_df.rename(columns={"Student ID": "StudentId", "prediction": "Target1"}, inplace = True)

In [24]:
final_df

## Vérification des différences entre régression logistique multinomial et arbre de décision.

In [20]:
df = pd.read_csv('/kaggle/input/model2/Model2.csv')

In [27]:
df2 = pd.concat([df,final_df],axis=1)
df2

In [45]:
lst = []
for i, row in df2.iterrows():
    if row['Target'] == row['Target1']:
        lst.append(1)
    else:
        lst.append(0)

df2['erreur'] = lst
df2

In [41]:
df2['erreur'].value_counts() 

In [43]:
df3 = pd.concat([df2,testing_probs],axis=1)
df3

In [ ]:
predicted_origins = testing_probs.idxmax(axis = 1)

In [ ]:
final_df.to_csv('mutliplereg.csv', index = False, sep = ',')